# **"CNN-Based Approach: AI-Powered Haircut Recommendation System via Facial Shape Recognition"**





# **Definition of the Problem**

The choice of a suitable haircut significantly impacts personal appearance and confidence. However, determining the right haircut often involves trial and error or subjective opinions, which can lead to unsatisfactory results. This project aims to solve this problem by using a CNN-based model to classify facial shapes from images and recommend suitable haircuts based on these classifications.

The key problem lies in automating facial shape recognition using CNNs, mapping facial shapes to haircuts, and ensuring that the system works across diverse facial structures and features.

# **Data Acquisition**
The dataset used for this project is sourced from Kaggle: Men Face Shape Dataset. This dataset contains images of male faces categorized into five facial shapes:
1. Oblong
2. Heart
3. Round
4. Square

**Dataset Details:**
* **Number of Images**: 1000 labeled images.
* **Features**: Images of male faces annotated with the corresponding face shape labels.
* **Diversity**: Includes variations in lighting, angles, and facial structures to improve model generalization.

**Steps for Acquisition:**
1. The dataset was downloaded directly from Kaggle.
2. Files were stored and organized into folders corresponding to the face shape labels for easier preprocessing and exploration.
3. Images were verified for quality and label accuracy before further processing.

# **Exploration and Analysis of Data**

**Understanding the Face Shape**


**Oblong Face Shape**
An Oblong face is characterized by a longer-than-average face length, with a narrow forehead and chin. Haircuts that add volume and width to the sides of the head work best for balancing the proportions. For example, haircuts such as the Classic Side Part, Pompadour, and Undercut can help create the illusion of a more proportionate face by adding height and structure without further elongating the face. The Buzz Cut and Classic Fade are also ideal for this shape because they maintain the face's clean lines while ensuring that the side volume does not overwhelm the face (HairLooks Blog: Face Shape Guide, n.d.).

**Heart Face Shape**
A Heart-shaped face has a wider forehead and a narrow chin, with the goal being to soften the sharp angles and create balance. Haircuts that provide volume near the chin and jawline work wonders here. The Classic Side Part and Long Quiff provide balance by adding volume to the top and side, while the Slick Back and Undercut help streamline the face. The Short Faux Hawk and Side Part Fringe soften the wider forehead and draw attention to the lower part of the face, making these haircuts great choices for a heart-shaped face (Boxin, 2024).

**Round Face Shape**
A Round face is characterized by soft, rounded edges and equal width and height. The goal for a round face is to create angles and elongate the face. High Volume Haircuts, such as the Pompadour, and Spiky Hairstyles, can add height and definition. The Neat Middle-Part Haircut and Skin Fade Hairstyle also work well to slim the appearance of the face. Additionally, Asymmetrical Styles and Crispy Cuts help create structure, making the face appear more angular and elongated.

**Square Face Shape**
A Square face features a strong, prominent jawline with equal width and height. To balance the sharp angles, softening the jawline with volume on top and at the sides is key. Haircuts such as the Full Shave, Mega-Pompadour, and Gelled to the Side help achieve this. The Undercut adds contrast and height while maintaining clean lines. Close-Cropped Curls and Gentleman’s Cut are also ideal as they soften the face and provide a stylish, structured look without exaggerating the jawline’s prominence (Charles-Philippe, 2022).


**Visualization of Data:**

A bar chart was created to show the number of images per facial shape. For instance:
* Oblong: 800 images
* Heart: 800 images
* Round: 800 images
* Square: 800 images

**Key Observations:**
1. Images vary in resolution, requiring standardization during preprocessing.
2. Lighting and background conditions are diverse, which is beneficial for model generalization.



# **Data Preparation**

**Steps for Preprocessing:**


1. **Resizing:** All images were resized to 224x224 pixels to standardize input dimensions for the CNN model.
2. **Normalization:** Pixel values were scaled to the range [0, 1] to facilitate faster convergence during model training.
3. **Label Encoding:** The face shape labels (e.g., Oval, Round) were encoded into numerical values for use in the CNN model.
  * Oblong: 0
  * Heart: 1
  * Round: 2
  * Square: 3
4. **Splitting Data:**
  * 80% of the images were allocated for training.
  * 20% were reserved for testing.

**Augmentation:**
To enhance the dataset, the following augmentation techniques were applied:
* **Rotation:** Randomly rotate images by up to 15 degrees.
* **Flipping:** Horizontally flip images.
* **Brightness Adjustment:** Randomly adjust brightness to simulate different lighting conditions.
* **Zooming:** Randomly zoom into images to focus on specific facial features.

**Dataset Statistics After Preparation:**
* **Training Set:** 3200 images
* **Testing Set:** 800 images

# **References**
HairLooks Blog: Face Shape Guide. (n.d.). HairLooks. https://blog.thehairlooks.com/blog-face-shape-guide.html

Boxin, B. (2024, July 11). Guide to Best Hairstyles for Men’s Face Shapes. The Official Blog of Hair Cuttery. https://blog.haircuttery.com/cut/best-hairstyles-for-mens-face-shapes/

Charles-Philippe. (2022, March 28). Men’s Square Face Shapes Guide: Best Hairstyles, Beards & More. https://bespokeunit.com/face-shapes/square/

In [ ]:
# Importing necessary libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Loading the Dataset
train_dir = 'C:\\Users\\Wawiworld\\Documents\\School\\CSC173_IntelligentSystems\\dataset\\training_set'
test_dir = 'C:\\Users\\Wawiworld\\Documents\\School\\CSC173_IntelligentSystems\\dataset\\testing_set'

# Checking the directory structure
train_oblong = os.listdir(f'{train_dir}/oblong')
train_heart = os.listdir(f'{train_dir}/heart')
train_round = os.listdir(f'{train_dir}/round')
train_square = os.listdir(f'{train_dir}/square')


In [ ]:
# Using ImageDataGenerator to load images and perform data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255) 

# Loading data using flow_from_directory method
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
# Check the class distribution
class_counts = {
    'Oblong': len(train_oblong),
    'Heart': len(train_heart),
    'Round': len(train_round),
    'Square': len(train_square)
}

# Display the distribution of samples
class_counts


In [ ]:
# Visualizing the distribution of samples in each category
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title('Distribution of Facial Shapes in Training Set')
plt.xlabel('Facial Shape')
plt.ylabel('Number of Images')
plt.show()


In [ ]:
# Function to display random images from the dataset
def display_random_images(directory, label, num_images=5):
    plt.figure(figsize=(15, 10))
    images = os.listdir(directory)
    for i in range(num_images):
        img_path = os.path.join(directory, images[i])
        img = load_img(img_path, target_size=(224, 224))
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.title(label)
        plt.axis('off')
    plt.show()

# Display random images for each facial shape category
display_random_images(f'{train_dir}/oblong', 'Oblong', 5)
display_random_images(f'{train_dir}/heart', 'Heart', 5)
display_random_images(f'{train_dir}/round', 'Round', 5)
display_random_images(f'{train_dir}/square', 'Square', 5)


In [ ]:
# Check for corrupt images by attempting to load and catch errors
def check_image_integrity(directory):
    corrupt_images = []
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    corrupt_images.append(img_path)
            except Exception as e:
                corrupt_images.append(img_path)
    return corrupt_images

corrupt_train_images = check_image_integrity(train_dir)
corrupt_test_images = check_image_integrity(test_dir)

print("Corrupt images in training data:", corrupt_train_images)
print("Corrupt images in testing data:", corrupt_test_images)


In [ ]:
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model using the training and validation data
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=[early_stopping]
)


In [ ]:
# Dictionary of haircuts for each facial shape
recommended_haircuts = {
    'Oblong': [
        'Classic side part', 'Faded side part', 'The long side part', 'Classic fade', 
        'Quiff', 'Undercut', 'Pompadour', 'Pompadour with a fade', 'The Ivy League', 'Buzz cut'
    ],
    'Heart': [
        'Side Part Fringe', 'The Classic Side Part', 'Long hair side part', 'The Long Fringe', 
        'The Classic Quiff', 'The Long Quiff', 'The Slick Back', 'Short Faux Hawk', 
        'Buzz Cut', 'The Undercut'
    ],
    'Round': [
        'Curly Hair Undercut Look', 'Neat Middle-Part Haircut', 'Skin Fade Hairstyle', 'Crispy Cut', 
        'High Volume Haircut', 'Asymmetrical Style', 'Spiky', 'Classic pompadour', 'Side part', 'Vertical haircut'
    ],
    'Square': [
        'Gelled to the Side', 'Full Shave', 'Mega-Pompadour', 'Bare-Guard Buzz', 'Line-Up Buzz', 
        'Receded Buzz', 'Close-Cropped Curls', 'Modern Flat Top', 'Undercut', 'Gentleman\'s Cut'
    ]
}


In [ ]:
# Load and preprocess a new image for prediction
img_path = 'dataset\\testing_set\\Square\\30000806490_500.jpg' 
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img) / 255.0 
img_array = np.expand_dims(img_array, axis=0)

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)

# Map the predicted class to the corresponding face shape
face_shape = {0: 'Oblong', 1: 'Heart', 2: 'Round', 3: 'Square'}
predicted_face_shape = face_shape[predicted_class[0]]

# Recommend haircuts based on the predicted face shape
recommended_list = recommended_haircuts[predicted_face_shape]

# Display the result
print(f"Predicted Face Shape: {predicted_face_shape}")
print(f"Recommended Haircuts: {', '.join(recommended_list)}")
